# Approaches for Network Learning Problems

There are many different ways to approach a network machine learning problem, and it is useful to have a general sense of what you might be thinking about when you're beginning a network data science project. Here is a set of things you should be thinking about in the beginning.

- What type of network you're working with
- Which part of the network you're investigating
- What kind of preprocessing you'll need to do
- How you're understanding the statistical process that generated your network
- How you're going to represent your network in a way that lets you investigate the properties you care about easily
- What you're actually going to do once you have a good representation of your network

Let's look at each of these criteria more closely.

## What type of network you're working with

We talked earlier about the different types of networks out there. Most of this book will use simple networks as examples: networks whose edges don't have a direction or weight. Sometimes we'll have multiple networks. We'll also run into situations where you have additional information, called covariates, to add to node-and-edge valued data you already have. 

There are other types of networks out there which we won't discuss in this book, since techniques that use these types of networks to learn from data are underdeveloped. Just so that you know they're out there, here are some of them. Most of the time, you can find a way to turn your complicated type of network into a simple network.

- **Bipartite networks** come into play when your nodes belong to two completely distinct classes, and connections can only be established between two members of different classes. For instance, let's say you're a stock trader and you're creating a big stock network, where the nodes are stocks and the edges are the degree of covariance in stock value between them. Now, let's say that on top of that simple (albeit weighted) network, you want to cluster stocks into groups: you have your energy sector group, your entertainment sector group, and so on. The kicker is that you want your groups to be nodes *themselves*, which have edges with all of the stocks you're interested in. If you just added these groups as nodes in the original network, you lose the fact that these group nodes are qualitatively different than the other types of nodes.

- **Multilayer networks** emerge when you have a common set of nodes, but each node has multiple *types* of edges. For instance, say you're tracking the same group of users on social media networks across platforms: you have a Twitter network and an Instagram friend network containing the same people. Between any pair of nodes, there are *two* possible edges: whether they're friends on Instagram, and whether they're friends on Twitter. Fortunately, there's a simple fix for this: you can just break multilayer networks apart into separate simple networks, one for Instagram and one for Twitter.

- **Hypergraphs** are networks whose edges don't necessarily only connect two nodes. For instance, say you're at a dance party and you've brought three close friends, all of whom know each other. In addition to the pairwise relationships between your friends, you've also got a group dynamic: the relationship between you, your friend Susan, and both of your friend Tom is its own distinct thing, regardless of your one-on-one relationships with each of them. This is called a hyperedge: an edge which connects more than two nodes together. In this case, there would be a single edge between you, Susan, and Tom. Usually, to turn hypergraphs into simple graphs, you'd turn hyperedges into pairwise edges that fully connect all of the nodes involved: the single hyperedge between you, Susan, and Tom would turn into three edges that form a triangle. This allows you to use a simple model in exchange for losing some information about group relationships.

- **Dynamic networks** change over time. Think of daily snapshots of a road network, where the nodes are streets and the edges are intersections. This network will mostly be the same every day, but on some of the days, subsets of intersections might have construction going on or accidents blocking the road, temporarily deleting the edge between those two streets. We'll work with dynamic networks a little bit in this book: the main way to deal with them is to just view each snapshot as its own network, and use methods which were created to work with multiple networks.

## Which part of the network you're investigating

Once you figure out what type of network you have, you'll want to figure out exactly what you're interested in learning about. Do you care about the nodes? The edges? Do you have multiple networks, or only one? We'll get into the weeds with this in chapter four. Right now, all you need to know is that there are four main avenues of investigation: treating nodes as the primary object, treating edges as the primary object, treating statistics you've calculated from your network as the primary objects, or viewing the network as a whole and treating it as a single object, to compare with another network (or networks). Most of the approaches in this book will use a node-based approach, but we'll touch on some edge-based and network-based approaches as well. We won't use calculated network statistics as much, for reasons we'll get into later.

In [1]:
# TODO: figure that shows bag of nodes, bag of edges, bag of features, bag of networks

## What kind of preprocessing you need to do

Although the majority of this book is about what to do after you have a fully formed network, it's important to think critically about what you need to do to a network to get it into a good form. For instance, how was the network data originally stored on your computer, and what format was it in? How noisy is your network, and is there anything you can do to clean up that noise? Are there isolated nodes which don't add any information? 

You'll get your first exposure to this in the end-to-end network machine learning project in chapter two, where you'll investigate a larval fly brain network, pulling the data from raw sources.

## Understanding the statistical process that generated your network

A key concept that you'll see throughout this book, especially if you are uninitiated in statistics, is viewing every network as simply an observation which resulted from a data-generating process, and what you're fundamentally doing is trying to reconstruct the properties of that data-generating process using your observation.

We'll get into the different types of ways you can model a data-generating process heavily in chapter five. For now, here's a brief summary of the types of models you might consider, given the network data that you observe.

### The Erdös-Rényi Model

The ER random network model is the simplest around, and is probably the most straightforward to understand. Under this model, for any two nodes in our network, there is some probability that they'll have an edge (say, $p = 0.3$). That means that when you observe a network whose generating process was Erdös-Rényi, it would be unlikely to find groups or communities, or even any structure at all, really. Erdös-Rényi networks have well-defined mathematical properties: we know how many edges on average the nodes should be expected to have, for instance.

### The Stochastic Block Model

An SBM is a step up in complexity from the ER model, and we'll be using it heavily throughout this book. It builds on top of the ER model in complexity: The basic idea is that, in addition to each edge being associated with a probability, each node is also associated with a community, and edge probabilities can change depending on the community your node belongs to. When we're trying to reconstruct community membership, we'll usually assume that our data-generating process happened as the result of some kind of SBM. 

### Random Dot Product Graphs

The ER model and the SBM can both be thought of as particular cases of an Random Dot Product Graph. The RDPG is a little bit harder-to-understand, since it uses a few concepts from linear algebra at its core, with the advantage of being much more dynamic. The basic idea is that, rather than being associated with a probability, each node is associated with a *latent position*: a vector with however many entries you want. The edge probability for a given pair of nodes is given by the dot product between their latent positions. In an ER model, every node has the same latent position; in an SBM, every node belonging to the same community has the same latent position.

When we find representations for networks throughout this book, an overarching theme will be assuming that the networks were drawn from an RDPG under the hood, and estimating these latent positions will be central to a large portion of its content.

### Multiple Network Models

Sometimes you'll be in a situation where, instead of having one network, you have more than one, and there's some type of dependency that makes it useful to have a single data-generating process whose data is *groups* of networks, rather than individual networks. There are a couple multiple-network models that you'll see in this group:
- The Joint RDPG model simply assumes that each of the networks it generated are independent and generated by the same latent positions. You're implictly assuming that all of your networks are RDPGs and have latent positions.
- The COSIE (Common Edge Independent Subspace) model is a bit more dynamic: it assumes that there is a shared overall structure between the data-generating process for all of your networks, but that there can be variation on a per-network basis. The details of this involve a fair amount of linear algebra, so we'll save it for chapter five. Just know that it exists.

### Network Models with Covariates

The last type of network model that you'll see in this book come into play when you have covariates, which we've already talked about a bit: an extra set of features for each node. 